In [1]:
# 当y、yhat、ycomp是从所有序列中取值组成的序列时，当在第一大类中出现（y-ycomp==0），不能直接区分没有补偿和完全准确补偿的情况；
# 当在第二大类中出现（y-yhat==0），不能直接区分没有预测和完全准确预测的情况；
# 当在第三大类中出现（y-yhat==y-ycomp==0），不能直接区分没有预测、没有补偿、完全预测、完全补偿这四种情况；
# 基于上述原因，会导致无法准确统计某些分项的指标；建议y、yhat、ycomp从所有发生补偿的序列中取值，将没有发生补偿的序列排除。

# import random
# import pandas as pd
# import numpy as np

# weights = []
# for i in range(-99,1):
#     weights.append((0.94)**i)

# k1, k2, k3 = 5000, 4000, 1100
# y = pd.Series(random.choices(range(0,100), k=k1, weights=weights))  # 生成随机序列，模拟k1个序列在某一周的真实销量

# z1 = pd.Series(10*np.sin(random.choices(range(0,5000), k=k2)))  # 假定k1个序列中只有k2个有预测销量
# z2 = pd.Series([0 for _ in range(k1-k2)])
# z3 = pd.concat([z1, z2])
# z4 = z3.sample(frac=1, replace=True)  # 打乱series的顺序
# z4.reset_index(drop=True, inplace=True)
# yhat = y + z4

# z5 = pd.Series(10*np.sin(random.choices(range(0,5000), k=k3)))  # 假定k1个序列中只有k3个发生补偿
# z6 = pd.Series([0 for _ in range(k1-k3)])
# z7 = pd.concat([z1, z2])
# z8 = z7.sample(frac=1, replace=True)  # 打乱series的顺序
# z8.reset_index(drop=True, inplace=True)
# ycomp = y + z8

In [2]:
import random
import pandas as pd
import numpy as np

weights = []
for i in range(-99,1):
    weights.append(0.94**i)

#  真实值组成的序列y、预测值组成的序列yhat、补偿值组成的序列ycomp，这三条序列组成中的序列是一一对应的，即同一序列在三条序列中的index相同
k1 = 5000
y = pd.Series(random.choices(range(0,100), k=k1, weights=weights))  # 生成随机序列，模拟5000个序列在某一周的真实销量

z1 = pd.Series(np.sin(random.choices(range(0,100), k=k1)))  # 假定这5000个是都有预测销量的序列
yhat = y + z1

z2 = pd.Series(np.sin(random.choices(range(0,100), k=k1)))  # 并且假定这5000个是都发生补偿的序列
ycomp = y + z2

In [3]:
# 得到经过第一层的三大类条件判断之后的真实值序列y_s1/2/3、预测值序列yhat_s1/2/3、补偿值序列ycomp_s1/2/3.

# 1. 补偿变好这一大类：
y_s1 = y[abs(y-yhat)>abs(y-ycomp)]  # 选出y序列中补偿变好的这一大类序列
yhat_s1 = yhat[abs(y-yhat)>abs(y-ycomp)]  # 选出yhat序列中补偿变好的这一大类序列
ycomp_s1 = ycomp[abs(y-yhat)>abs(y-ycomp)]  # 选出ycomp序列中补偿变好的这一大类序列

# 2. 补偿变差这一大类：
y_s2 = y[abs(y-yhat)<abs(y-ycomp)]  # 选出y序列中补偿变差的这一大类序列
yhat_s2 = yhat[abs(y-yhat)<abs(y-ycomp)]  # 选出yhat序列中补偿变差的这一大类序列
ycomp_s2 = ycomp[abs(y-yhat)<abs(y-ycomp)]  # 选出ycomp序列中补偿变差的这一大类序列

# 3. 补偿无效果这一大类：分为1.发生补偿但补偿量恰好为零，2.恰好补偿到真实值的对侧等距点；
# 其中1.又分为：1.1 真实值、预测值、补偿值恰好相等，1.2 预测值与补偿值相等，但与真实值不等。
# 由于满足第三大类条件的序列很少，所以暂不对其进行细分。
y_s3 = y[abs(y-yhat)==abs(y-ycomp)]  # 选出y序列中补偿无效果的这一大类序列
# yhat_s3 = yhat[abs(y-yhat)==abs(y-ycomp)]  # 选出yhat序列中补偿无效果的这一大类序列
# ycomp_s3 = ycomp[abs(y-yhat)==abs(y-ycomp)]  # 选出ycomp序列中补偿无效果的这一大类序列

In [4]:
if len(y) == len(yhat) == len(ycomp):  # 若该条件满足，则必有：len(y_s1)==len(yhat_s1)==len(ycomp_s1), len(y_s2)==len(yhat_s2)==len(ycomp_s2), len(y_s3)==len(yhat_s3)==len(ycomp_s3).
    # 因为这里只需获取三大类情况各自的序列数量m1/m2/m3，所以从y、yhat、ycomp任一序列中选取序列，得到的m1/m2/m3都相等。下面是从y序列中选取序列，得到的yhat序列的长度m1/m2/m3.
    m1, m2, m3 = len(y_s1), len(y_s2), len(y_s3)
    mt = m1+m2+m3
    if mt == k1:
        x1, x2, x3 = m1/mt*100, m2/mt*100, m3/mt*100
        print('计算正确，补偿变好、变坏、无效果三种情况的序列数相加与发生补偿的序列总数相等：\n')
        print('补偿变好的序列数有：{:.0f}个，占全部发生补偿的序列百分比：{:.2f}%'.format(m1, x1))
        print('补偿变差的序列数有：{:.0f}个，占全部发生补偿的序列百分比：{:.2f}%'.format(m2, x2))
        print('补偿无效果的序列数有：{:.0f}个，占全部发生补偿的序列百分比：{:.2f}%'.format(m3, x3))
    else: raise Exception('计算有误，变好、变化、无效果三种情况的序列数相加与发生补偿的序列总数不等。')
else: raise Exception('真实值、预测值、补偿值组成的序列长度不等，建议化为等长，以避免后续某些指标代表的成分不唯一的情况出现。')


计算正确，补偿变好、变坏、无效果三种情况的序列数相加与发生补偿的序列总数相等：

补偿变好的序列数有：2488个，占全部发生补偿的序列百分比：49.76%
补偿变差的序列数有：2468个，占全部发生补偿的序列百分比：49.36%
补偿无效果的序列数有：44个，占全部发生补偿的序列百分比：0.88%


In [5]:
# 在经过第一层条件判断之后，得到经过第二层条件判断的各个真实值序列、预测值序列、补偿值序列


# 对于补偿变好这一大类中的6小类情况：

# 1.1 预测补偿均低，补偿同侧变好：
y_s1_c1 = y_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1>0)]
yhat_s1_c1 = yhat_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1>0)]
ycomp_s1_c1 = ycomp_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1>0)]
# 1.2 预测补偿均高，补偿同侧变好：
y_s1_c2 = y_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1<0)]
yhat_s1_c2 = yhat_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1<0)]
ycomp_s1_c2 = ycomp_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1<0)]
# 1.3 预低补高，补偿对侧变好：
y_s1_c3 = y_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1<0)]
yhat_s1_c3 = yhat_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1<0)]
ycomp_s1_c3 = ycomp_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1<0)]
# 1.4 预高补低，补偿对侧变好：
y_s1_c4 = y_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1>0)]
yhat_s1_c4 = yhat_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1>0)]
ycomp_s1_c4 = ycomp_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1>0)]
# 1.5 预低，完全补偿：
y_s1_c5 = y_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1==0)]
yhat_s1_c5 = yhat_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1==0)]
ycomp_s1_c5 = ycomp_s1[(y_s1-yhat_s1>0)&(y_s1-ycomp_s1==0)]
# 1.6 预高，完全补偿：
y_s1_c6 = y_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1==0)]
yhat_s1_c6 = yhat_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1==0)]
ycomp_s1_c6 = ycomp_s1[(y_s1-yhat_s1<0)&(y_s1-ycomp_s1==0)]


# 对于补偿变差这一大类中的6小类情况：

# 2.1 预测补偿均低，补偿同侧变差：
y_s2_c1 = y_s2[(y_s2-yhat_s2>0)&(y_s2-ycomp_s2>0)]
yhat_s2_c1 = yhat_s2[(y_s2-yhat_s2>0)&(y_s2-ycomp_s2>0)]
ycomp_s2_c1 = ycomp_s2[(y_s2-yhat_s2>0)&(y_s2-ycomp_s2>0)]
# 2.2 预测补偿均高，补偿同侧变差：
y_s2_c2 = y_s2[(y_s2-yhat_s2<0)&(y_s2-ycomp_s2<0)]
yhat_s2_c2 = yhat_s2[(y_s2-yhat_s2<0)&(y_s2-ycomp_s2<0)]
ycomp_s2_c2 = ycomp_s2[(y_s2-yhat_s2<0)&(y_s2-ycomp_s2<0)]
# 2.3 预低补高，补偿对侧变差：
y_s2_c3 = y_s2[(y_s2-yhat_s2>0)&(y_s2-ycomp_s2<0)]
yhat_s2_c3 = yhat_s2[(y_s2-yhat_s2>0)&(y_s2-ycomp_s2<0)]
ycomp_s2_c3 = ycomp_s2[(y_s2-yhat_s2>0)&(y_s2-ycomp_s2<0)]
# 2.4 预高补低，补偿对侧变差：
y_s2_c4 = y_s2[(y_s2-yhat_s2<0)&(y_s2-ycomp_s2>0)]
yhat_s2_c4 = yhat_s2[(y_s2-yhat_s2<0)&(y_s2-ycomp_s2>0)]
ycomp_s2_c4 = ycomp_s2[(y_s2-yhat_s2<0)&(y_s2-ycomp_s2>0)]
# 2.5 完全准确预测，补偿偏低：
y_s2_c5 = y_s2[(y_s2-yhat_s2==0)&(y_s2-ycomp_s2>0)]
yhat_s2_c5 = yhat_s2[(y_s2-yhat_s2==0)&(y_s2-ycomp_s2>0)]
ycomp_s2_c5 = ycomp_s2[(y_s2-yhat_s2==0)&(y_s2-ycomp_s2>0)]
# 2.6 完全准确预测，补偿偏高：
y_s2_c6 = y_s2[(y_s2-yhat_s2==0)&(y_s2-ycomp_s2<0)]
yhat_s2_c6 = yhat_s2[(y_s2-yhat_s2==0)&(y_s2-ycomp_s2<0)]
ycomp_s2_c6 = ycomp_s2[(y_s2-yhat_s2==0)&(y_s2-ycomp_s2<0)]

In [6]:
# 1. 对于第一大类即补偿变好的情况

m11, m12 = len(y_s1_c1), len(y_s1_c2)  # 同侧补偿且变好
m13, m14 = len(y_s1_c3), len(y_s1_c4)  # 对侧补偿且变好
m15, m16 = len(y_s1_c5), len(y_s1_c6)  # 完全补偿
m1syn, m1opp, m1com = m11+m12, m13+m14, m15+m16  # 补偿变好时同侧、对侧、完全补偿的序列数
if m11+m12+m13+m14+m15+m16 == m1:
    x11, x12, x13, x14, x15, x16 = m11/m1*100, m12/m1*100, m13/m1*100, m14/m1*100, m15/m1*100, m16/m1*100
    x1syn, x1opp, x1com = m1syn/m1*100, m1opp/m1*100, m1com/m1*100
    print('计算正确，当补偿变好时，其下6种子情况的序列总数相加与补偿变好的序列总数相等：\n')
    print('预测销量与补偿后销量均比真实销量低，补偿变好且发生在同侧，该种情况有：{:.0f}个序列，\
占补偿变好序列数的百分比为：{:.2f}%'.format(m11, x11))
    print('预测销量与补偿后销量均比真实销量高，补偿变好且发生在同侧，该种情况有：{:.0f}个序列，\
占补偿变好序列数的百分比为：{:.2f}%'.format(m12, x12))
    print('预测销量偏低，补偿后销量偏高，补偿变好且发生在对侧，该种情况有：{:.0f}个序列，\
占补偿变好序列数的百分比为：{:.2f}%'.format(m13, x13))
    print('预测销量偏高，补偿后销量偏低，补偿变好且发生在对侧，该种情况有：{:.0f}个序列，\
占补偿变好序列数的百分比为：{:.2f}%'.format(m14, x14))
    print('预测销量偏低，完全补偿，该种情况有：{:.0f}个序列，占补偿变好序列数的百分比为：{:.2f}%'.format(m15, x15))
    print('预测销量偏高，完全补偿，该种情况有：{:.0f}个序列，占补偿变好序列数的百分比为：{:.2f}%'.format(m16, x16))
    print('补偿变好且发生在同侧的序列数有：{:.0f}个，占补偿变好序列数的百分比为：{:.2f}%'.format(m1syn, x1syn))
    print('补偿变好且发生在对侧的序列数有：{:.0f}个，占补偿变好序列数的百分比为：{:.2f}%'.format(m1opp, x1opp))
    print('完全补偿的序列数有：{:.0f}个，占补偿变好序列数的百分比为：{:.2f}%'.format(m1com, x1com))  
else: raise Exception('计算有误，当补偿变好时，其下6种子情况的序列总数相加与补偿变好的序列总数不等。')

计算正确，当补偿变好时，其下6种子情况的序列总数相加与补偿变好的序列总数相等：

预测销量与补偿后销量均比真实销量低，补偿变好且发生在同侧，该种情况有：615个序列，占补偿变好序列数的百分比为：24.72%
预测销量与补偿后销量均比真实销量高，补偿变好且发生在同侧，该种情况有：604个序列，占补偿变好序列数的百分比为：24.28%
预测销量偏低，补偿后销量偏高，补偿变好且发生在对侧，该种情况有：610个序列，占补偿变好序列数的百分比为：24.52%
预测销量偏高，补偿后销量偏低，补偿变好且发生在对侧，该种情况有：606个序列，占补偿变好序列数的百分比为：24.36%
预测销量偏低，完全补偿，该种情况有：23个序列，占补偿变好序列数的百分比为：0.92%
预测销量偏高，完全补偿，该种情况有：30个序列，占补偿变好序列数的百分比为：1.21%
补偿变好且发生在同侧的序列数有：1219个，占补偿变好序列数的百分比为：49.00%
补偿变好且发生在对侧的序列数有：1216个，占补偿变好序列数的百分比为：48.87%
完全补偿的序列数有：53个，占补偿变好序列数的百分比为：2.13%


In [7]:
# 1.1 预测补偿均低，补偿同侧变好：
print('预测补偿均低，补偿同侧变好时：\n')

pva_s1_c1 = abs(y_s1_c1 - yhat_s1_c1)  # 预测偏差量
pva_s1_c1_sts = pva_s1_c1.describe()
pvr_s1_c1 = abs(pva_s1_c1[y_s1_c1!=0] / y_s1_c1[y_s1_c1!=0])*100  # 预测偏差量与真实销量百分比，在此时的条件下，真实值(分母)为0的序列不能参与该指标计算，否则会使下面describe的统计指标失真。所以pvr_s1_c1计算的是真实销量不为0的那些序列。
pvr_s1_c1_sts = pvr_s1_c1.describe()
print('预测偏差量的统计指标：')
print(pva_s1_c1_sts)
print('预测偏差量与真实销量百分比的统计指标：')
print(pvr_s1_c1_sts)
print('----------------------------------------')

cva_s1_c1 = abs(y_s1_c1 - ycomp_s1_c1)  # 补偿后剩余偏差量
cva_s1_c1_sts = cva_s1_c1.describe()
cvr_s1_c1 = abs(cva_s1_c1[y_s1_c1!=0] / y_s1_c1[y_s1_c1!=0])*100  # 补偿后剩余偏差量与真实销量百分比，在此时的条件下，真实值(分母)为0的序列不能参与该指标计算，否则会使下面describe的统计指标失真。所以cvr_s1_c1计算的是真实销量不为0的那些序列。
cvr_s1_c1_sts = cvr_s1_c1.describe()
print('补偿后剩余偏差量的统计指标：')
print(cva_s1_c1_sts)
print('补偿后剩余偏差量与真实销量百分比的统计指标：')
print(cvr_s1_c1_sts)
print('----------------------------------------')

ca_s1_c1 = abs(yhat_s1_c1 - ycomp_s1_c1)  # 补偿量
ca_s1_c1_sts = ca_s1_c1.describe()
car_s1_c1 = abs(ca_s1_c1[pva_s1_c1!=0] / pva_s1_c1[pva_s1_c1!=0])*100  # 补偿量占预测偏差量百分比
car_s1_c1_sts = car_s1_c1.describe()
print('补偿量的统计指标：')
print(ca_s1_c1_sts)
print('补偿量占预测偏差量百分比的统计指标：')
print(car_s1_c1_sts)
print('----------------------------------------')

if (len(pva_s1_c1)==len(cva_s1_c1)==len(ca_s1_c1)==len(car_s1_c1)==m11) & (len(pva_s1_c1)>=len(pvr_s1_c1)) & (len(cva_s1_c1)>=len(cvr_s1_c1)): # 按位与运算符(&)的优先级高于比较运算符和等于运算符，所以需用()括起每个条件
    print('在1.1即预测补偿均低，补偿同侧变好的情况中，各分量的长度匹配正确。')
else: raise Exception('在1.1即预测补偿均低，补偿同侧变好的情况中，各分量的长度匹配存在错误，请检查该异常。')


预测补偿均低，补偿同侧变好时：

预测偏差量的统计指标：
count    615.000000
mean       0.817994
std        0.212153
min        0.026551
25%        0.750987
50%        0.905578
75%        0.985146
max        0.999990
dtype: float64
预测偏差量与真实销量百分比的统计指标：
count    585.000000
mean      14.440248
std       19.921333
min        0.139743
25%        3.331441
50%        7.129333
75%       15.390974
max       99.975517
dtype: float64
----------------------------------------
补偿后剩余偏差量的统计指标：
count    615.000000
mean       0.460380
std        0.290810
min        0.008851
25%        0.245252
50%        0.428183
75%        0.739181
max        0.999207
dtype: float64
补偿后剩余偏差量与真实销量百分比的统计指标：
count    585.000000
mean       8.192009
std       14.210647
min        0.013830
25%        1.354428
50%        3.400132
75%        8.191170
max       96.139749
dtype: float64
----------------------------------------
补偿量的统计指标：
count    615.000000
mean       0.357614
std        0.259748
min        0.000548
25%        0.130467
50%        0.318947
7

In [8]:
# 1.2 预测补偿均高，补偿同侧变好：
print('预测补偿均高，补偿同侧变好时：\n')

pva_s1_c2 = abs(y_s1_c2 - yhat_s1_c2)  # 预测偏差量
pva_s1_c2_sts = pva_s1_c2.describe()
pvr_s1_c2 = abs(pva_s1_c2[y_s1_c2!=0] / y_s1_c2[y_s1_c2!=0])*100  # 预测偏差量与真实销量百分比
pvr_s1_c2_sts = pvr_s1_c2.describe()
print('预测偏差量的统计指标：')
print(pva_s1_c2_sts)
print('预测偏差量与真实销量百分比的统计指标：')
print(pvr_s1_c2_sts)
print('----------------------------------------')

cva_s1_c2 = abs(y_s1_c2 - ycomp_s1_c2)  # 补偿后剩余偏差量
cva_s1_c2_sts = cva_s1_c2.describe()
cvr_s1_c2 = abs(cva_s1_c2[y_s1_c2!=0] / y_s1_c2[y_s1_c2!=0])*100  # 补偿后剩余偏差量与真实销量百分比
cvr_s1_c2_sts = cvr_s1_c2.describe()
print('补偿后剩余偏差量的统计指标：')
print(cva_s1_c2_sts)
print('补偿后剩余偏差量与真实销量百分比的统计指标：')
print(cvr_s1_c2_sts)
print('----------------------------------------')

ca_s1_c2 = abs(yhat_s1_c2 - ycomp_s1_c2)  # 补偿量
ca_s1_c2_sts = ca_s1_c2.describe()
car_s1_c2 = abs(ca_s1_c2[pva_s1_c2!=0] / pva_s1_c2[pva_s1_c2!=0])*100  # 补偿量占预测偏差量百分比
car_s1_c2_sts = car_s1_c2.describe()
print('补偿量的统计指标：')
print(ca_s1_c2_sts)
print('补偿量占预测偏差量百分比的统计指标：')
print(car_s1_c2_sts)
print('----------------------------------------')

if (len(pva_s1_c2)==len(cva_s1_c2)==len(ca_s1_c2)==len(car_s1_c2)==m12) & (len(pva_s1_c2)>=len(pvr_s1_c2)) & (len(cva_s1_c2)>=len(cvr_s1_c2)): # 按位与运算符(&)的优先级高于比较运算符和等于运算符，所以需用()括起每个条件
    print('在1.2即预测补偿均高，补偿同侧变好的情况中，各分量的长度匹配正确。')
else: raise Exception('在1.2即预测补偿均高，补偿同侧变好的情况中，各分量的长度匹配存在错误，请检查该异常。')


预测补偿均高，补偿同侧变好时：

预测偏差量的统计指标：
count    604.000000
mean       0.803461
std        0.228314
min        0.035398
25%        0.683262
50%        0.901788
75%        0.983588
max        0.999912
dtype: float64
预测偏差量与真实销量百分比的统计指标：
count    564.000000
mean      14.447944
std       20.538999
min        0.153906
25%        3.420898
50%        6.887246
75%       15.353737
max       99.991186
dtype: float64
----------------------------------------
补偿后剩余偏差量的统计指标：
count    604.000000
mean       0.476495
std        0.295373
min        0.017702
25%        0.232206
50%        0.436165
75%        0.736171
max        0.999520
dtype: float64
补偿后剩余偏差量与真实销量百分比的统计指标：
count    564.000000
mean       8.552893
std       15.155179
min        0.032781
25%        1.513497
50%        3.532695
75%        8.009499
max       98.358775
dtype: float64
----------------------------------------
补偿量的统计指标：
count    604.000000
mean       0.326966
std        0.257585
min        0.000392
25%        0.111599
50%        0.257338
7

In [9]:
# 1.3 预测偏低补偿偏高，补偿对侧变好：
print('预测偏低补偿偏高，补偿对侧变好时：\n')

pva_s1_c3 = abs(y_s1_c3 - yhat_s1_c3)  # 预测偏差量
pva_s1_c3_sts = pva_s1_c3.describe()
pvr_s1_c3 = abs(pva_s1_c3[y_s1_c3!=0] / y_s1_c3[y_s1_c3!=0])*100  # 预测偏差量与真实销量百分比
pvr_s1_c3_sts = pvr_s1_c3.describe()
print('预测偏差量的统计指标：')
print(pva_s1_c3_sts)
print('预测偏差量与真实销量百分比的统计指标：')
print(pvr_s1_c3_sts)
print('----------------------------------------')

cva_s1_c3 = abs(y_s1_c3 - ycomp_s1_c3)  # 补偿后剩余偏差量
cva_s1_c3_sts = cva_s1_c3.describe()
cvr_s1_c3 = abs(cva_s1_c3[y_s1_c3!=0] / y_s1_c3[y_s1_c3!=0])*100  # 补偿后剩余偏差量与真实销量百分比
cvr_s1_c3_sts = cvr_s1_c3.describe()
print('补偿后剩余偏差量的统计指标：')
print(cva_s1_c3_sts)
print('补偿后剩余偏差量与真实销量百分比的统计指标：')
print(cvr_s1_c3_sts)
print('----------------------------------------')

ca_s1_c3 = abs(yhat_s1_c3 - ycomp_s1_c3)  # 补偿量
ca_s1_c3_sts = ca_s1_c3.describe()
car_s1_c3 = abs(ca_s1_c3[pva_s1_c3!=0] / pva_s1_c3[pva_s1_c3!=0])*100  # 补偿量占预测偏差量百分比
car_s1_c3_sts = car_s1_c3.describe()
print('补偿量的统计指标：')
print(ca_s1_c3_sts)
print('补偿量占预测偏差量百分比的统计指标：')
print(car_s1_c3_sts)
print('----------------------------------------')

if (len(pva_s1_c3)==len(cva_s1_c3)==len(ca_s1_c3)==len(car_s1_c3)==m13) & (len(pva_s1_c3)>=len(pvr_s1_c3)) & (len(cva_s1_c3)>=len(cvr_s1_c3)): # 按位与运算符(&)的优先级高于比较运算符和等于运算符，所以需用()括起每个条件
    print('在1.3即预测偏低补偿偏高，补偿对侧变好的情况中，各分量的长度匹配正确。')
else: raise Exception('在1.3即预测偏低补偿偏高，补偿对侧变好的情况中，各分量的长度匹配存在错误，请检查该异常。')


预测偏低补偿偏高，补偿对侧变好时：

预测偏差量的统计指标：
count    610.000000
mean       0.806749
std        0.228044
min        0.114785
25%        0.739181
50%        0.905578
75%        0.985146
max        0.999990
dtype: float64
预测偏差量与真实销量百分比的统计指标：
count    579.000000
mean      13.815354
std       19.369675
min        0.279963
25%        3.102605
50%        6.249939
75%       14.183339
max       99.999021
dtype: float64
----------------------------------------
补偿后剩余偏差量的统计指标：
count    610.000000
mean       0.485275
std        0.294672
min        0.017702
25%        0.253823
50%        0.475072
75%        0.745113
max        0.992873
dtype: float64
补偿后剩余偏差量与真实销量百分比的统计指标：
count    579.000000
mean       8.396308
std       13.297504
min        0.039338
25%        1.247514
50%        3.522425
75%        8.377865
max       91.294525
dtype: float64
----------------------------------------
补偿量的统计指标：
count    610.000000
mean       1.292025
std        0.461563
min        0.132487
25%        0.995240
50%        1.353768

In [10]:
# 1.4 预测偏高补偿偏低，补偿对侧变好：
print('预测偏高补偿偏低，补偿对侧变好时：\n')

pva_s1_c4 = abs(y_s1_c4 - yhat_s1_c4)  # 预测偏差量
pva_s1_c4_sts = pva_s1_c4.describe()
pvr_s1_c4 = abs(pva_s1_c4[y_s1_c4!=0] / y_s1_c4[y_s1_c4!=0])*100  # 预测偏差量与真实销量百分比
pvr_s1_c4_sts = pvr_s1_c4.describe()
print('预测偏差量的统计指标：')
print(pva_s1_c4_sts)
print('预测偏差量与真实销量百分比的统计指标：')
print(pvr_s1_c4_sts)
print('----------------------------------------')

cva_s1_c4 = abs(y_s1_c4 - ycomp_s1_c4)  # 补偿后剩余偏差量
cva_s1_c4_sts = cva_s1_c4.describe()
cvr_s1_c4 = abs(cva_s1_c4[y_s1_c4!=0] / y_s1_c4[y_s1_c4!=0])*100  # 补偿后剩余偏差量与真实销量百分比
cvr_s1_c4_sts = cvr_s1_c4.describe()
print('补偿后剩余偏差量的统计指标：')
print(cva_s1_c4_sts)
print('补偿后剩余偏差量与真实销量百分比的统计指标：')
print(cvr_s1_c4_sts)
print('----------------------------------------')

ca_s1_c4 = abs(yhat_s1_c4 - ycomp_s1_c4)  # 补偿量
ca_s1_c4_sts = ca_s1_c4.describe()
car_s1_c4 = abs(ca_s1_c4[pva_s1_c4!=0] / pva_s1_c4[pva_s1_c4!=0])*100  # 补偿量占预测偏差量百分比
car_s1_c4_sts = car_s1_c4.describe()
print('补偿量的统计指标：')
print(ca_s1_c4_sts)
print('补偿量占预测偏差量百分比的统计指标：')
print(car_s1_c4_sts)
print('----------------------------------------')

if (len(pva_s1_c4)==len(cva_s1_c4)==len(ca_s1_c4)==len(car_s1_c4)==m14) & (len(pva_s1_c4)>=len(pvr_s1_c4)) & (len(cva_s1_c4)>=len(cvr_s1_c4)): # 按位与运算符(&)的优先级高于比较运算符和等于运算符，所以需用()括起每个条件
    print('在1.4即预测偏高补偿偏低，补偿对侧变好的情况中，各分量的长度匹配正确。')
else: raise Exception('在1.4即预测偏高补偿偏低，补偿对侧变好的情况中，各分量的长度匹配存在错误，请检查该异常。')


预测偏高补偿偏低，补偿对侧变好时：

预测偏差量的统计指标：
count    606.000000
mean       0.825766
std        0.204580
min        0.035398
25%        0.733190
50%        0.912945
75%        0.986628
max        0.999912
dtype: float64
预测偏差量与真实销量百分比的统计指标：
count    569.000000
mean      15.489402
std       20.553950
min        0.282787
25%        3.537883
50%        7.075767
75%       18.185949
max       99.991186
dtype: float64
----------------------------------------
补偿后剩余偏差量的统计指标：
count    606.000000
mean       0.488149
std        0.288390
min        0.008851
25%        0.249533
50%        0.444113
75%        0.750987
max        0.999207
dtype: float64
补偿后剩余偏差量与真实销量百分比的统计指标：
count    569.000000
mean       9.407896
std       14.813971
min        0.052067
25%        1.442024
50%        3.841273
75%       10.160354
max       91.652155
dtype: float64
----------------------------------------
补偿量的统计指标：
count    606.000000
mean       1.313915
std        0.426020
min        0.044250
25%        1.016817
50%        1.339619

In [11]:
# 1.5 预测偏低，完全补偿：
print('预测偏低，完全补偿时：\n')

pva_s1_c5 = abs(y_s1_c5 - yhat_s1_c5)  # 预测偏差量
pva_s1_c5_sts = pva_s1_c5.describe()
pvr_s1_c5 = abs(pva_s1_c5[y_s1_c5!=0] / y_s1_c5[y_s1_c5!=0])*100  # 预测偏差量与真实销量百分比
pvr_s1_c5_sts = pvr_s1_c5.describe()
print('预测偏差量的统计指标：')
print(pva_s1_c5_sts)
print('预测偏差量与真实销量百分比的统计指标：')
print(pvr_s1_c5_sts)
print('----------------------------------------')

cva_s1_c5 = abs(y_s1_c5 - ycomp_s1_c5)  # 补偿后剩余偏差量
cva_s1_c5_sts = cva_s1_c5.describe()
cvr_s1_c5 = abs(cva_s1_c5[y_s1_c5!=0] / y_s1_c5[y_s1_c5!=0])*100  # 补偿后剩余偏差量与真实销量百分比
cvr_s1_c5_sts = cvr_s1_c5.describe()
if ((cva_s1_c5==0).sum() == len(cva_s1_c5)) & ((cvr_s1_c5==0).sum() == len(cvr_s1_c5)):
    print('补偿后剩余偏差量的统计指标：')
    print(cva_s1_c5_sts)
    print('补偿后剩余偏差量与真实销量百分比的统计指标：')
    print(cvr_s1_c5_sts)
    print('----------------------------------------')
else: raise Exception('1.5 在第一大类的第五小类中，即预测偏低，完全补偿这种情况下，\
补偿偏差量(cva_s1_c5)和/或补偿偏差比(cvr_s1_c5)存在计算有误的情况，请检查错误原因。')

ca_s1_c5 = abs(yhat_s1_c5 - ycomp_s1_c5)  # 补偿量
ca_s1_c5_sts = ca_s1_c5.describe()
car_s1_c5 = abs(ca_s1_c5[pva_s1_c5!=0] / pva_s1_c5[pva_s1_c5!=0])*100  # 补偿量占预测偏差量百分比
car_s1_c5_sts = car_s1_c5.describe()
if ((ca_s1_c5==pva_s1_c5).sum() == len(ca_s1_c5)) & ((car_s1_c5==100).sum() == len(car_s1_c5)):
    print('补偿量的统计指标：')
    print(ca_s1_c5_sts)
    print('补偿量占预测偏差量百分比的统计指标：')
    print(car_s1_c5_sts)
    print('----------------------------------------')
else: raise Exception('1.5 在第一大类的第五小类中，即预测偏低，完全补偿这种情况下，\
补偿量(ca_s1_c5)、补偿比(car_s1_c5)、预测偏差量(pva_s1_c5)中存在计算有误的情况，请检查错误原因。')

if (len(pva_s1_c5)==len(cva_s1_c5)==len(ca_s1_c5)==len(car_s1_c5)==m15) & (len(pva_s1_c5)>=len(pvr_s1_c5)) & (len(cva_s1_c5)>=len(cvr_s1_c5)): # 按位与运算符(&)的优先级高于比较运算符和等于运算符，所以需用()括起每个条件
    print('在1.5即预测偏低完全补偿的情况中，各分量的长度匹配正确。')
else: raise Exception('在1.5即预测偏低完全补偿的情况中，各分量的长度匹配存在错误，请检查该异常。')


预测偏低，完全补偿时：

预测偏差量的统计指标：
count    23.000000
mean      0.742712
std       0.264167
min       0.158623
25%       0.656281
50%       0.779466
75%       0.977075
max       0.999990
dtype: float64
预测偏差量与真实销量百分比的统计指标：
count    23.000000
mean      7.237119
std       8.549974
min       0.755346
25%       3.242788
50%       5.045350
75%       6.266893
max      38.973303
dtype: float64
----------------------------------------
补偿后剩余偏差量的统计指标：
count    23.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
dtype: float64
补偿后剩余偏差量与真实销量百分比的统计指标：
count    23.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
dtype: float64
----------------------------------------
补偿量的统计指标：
count    23.000000
mean      0.742712
std       0.264167
min       0.158623
25%       0.656281
50%       0.779466
75%       0.977075
max       0.999990
dtype: float64
补偿量占预测偏差量百分比的统计指标：
count     23.0
mean     100.0
std        0.0
min   

In [12]:
# 1.6 预测偏高，完全补偿：
print('预测偏高，完全补偿时：\n')

pva_s1_c6 = abs(y_s1_c6 - yhat_s1_c6)  # 预测偏差量
pva_s1_c6_sts = pva_s1_c6.describe()
pvr_s1_c6 = abs(pva_s1_c6[y_s1_c6!=0] / y_s1_c6[y_s1_c6!=0])*100  # 预测偏差量与真实销量百分比
pvr_s1_c6_sts = pvr_s1_c6.describe()
print('预测偏差量的统计指标：')
print(pva_s1_c6_sts)
print('预测偏差量与真实销量百分比的统计指标：')
print(pvr_s1_c6_sts)
print('----------------------------------------')

cva_s1_c6 = abs(y_s1_c6 - ycomp_s1_c6)  # 补偿后剩余偏差量
cva_s1_c6_sts = cva_s1_c6.describe()
cvr_s1_c6 = abs(cva_s1_c6[y_s1_c6!=0] / y_s1_c6[y_s1_c6!=0])*100  # 补偿后剩余偏差量与真实销量百分比
cvr_s1_c6_sts = cvr_s1_c6.describe()
if ((cva_s1_c6==0).sum() == len(cva_s1_c6)) & ((cvr_s1_c6==0).sum() == len(cvr_s1_c6)):
    print('补偿后剩余偏差量的统计指标：')
    print(cva_s1_c6_sts)
    print('补偿后剩余偏差量与真实销量百分比的统计指标：')
    print(cvr_s1_c6_sts)
    print('----------------------------------------')
else: raise Exception('1.6 在第一大类的第六小类中，即预测偏高，完全补偿这种情况下，\
补偿偏差量(cva_s1_c6)和/或补偿偏差比(cvr_s1_c6)存在计算有误的情况，请检查错误原因。')

ca_s1_c6 = abs(yhat_s1_c6 - ycomp_s1_c6)  # 补偿量
ca_s1_c6_sts = ca_s1_c6.describe()
car_s1_c6 = abs(ca_s1_c6[pva_s1_c6!=0] / pva_s1_c6[pva_s1_c6!=0])*100  # 补偿量占预测偏差量百分比
car_s1_c6_sts = car_s1_c6.describe()
if ((ca_s1_c6==pva_s1_c6).sum() == len(ca_s1_c6)) & ((car_s1_c6==100).sum() == len(car_s1_c6)):
    print('补偿量的统计指标：')
    print(ca_s1_c6_sts)
    print('补偿量占预测偏差量百分比的统计指标：')
    print(car_s1_c6_sts)
    print('----------------------------------------')
else: raise Exception('1.6 在第一大类的第六小类中，即预测偏高，完全补偿这种情况下，\
补偿量(ca_s1_c6)、补偿比(car_s1_c6)、预测偏差量(pva_s1_c6)中存在计算有误的情况，请检查错误原因。')

if (len(pva_s1_c6)==len(cva_s1_c6)==len(ca_s1_c6)==len(car_s1_c6)==m16) & (len(pva_s1_c6)>=len(pvr_s1_c6)) & (len(cva_s1_c6)>=len(cvr_s1_c6)): # 按位与运算符(&)的优先级高于比较运算符和等于运算符，所以需用()括起每个条件
    print('在1.6即预测偏高完全补偿的情况中，各分量的长度匹配正确。')
else: raise Exception('在1.6即预测偏高完全补偿的情况中，各分量的长度匹配存在错误，请检查该异常。')


预测偏高，完全补偿时：

预测偏差量的统计指标：
count    30.000000
mean      0.751059
std       0.245066
min       0.123573
25%       0.592138
50%       0.843780
75%       0.912945
max       0.999912
dtype: float64
预测偏差量与真实销量百分比的统计指标：
count    27.000000
mean     19.522640
std      25.142862
min       0.826692
25%       3.011883
50%      10.143836
75%      22.523666
max      91.294525
dtype: float64
----------------------------------------
补偿后剩余偏差量的统计指标：
count    30.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
dtype: float64
补偿后剩余偏差量与真实销量百分比的统计指标：
count    27.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
dtype: float64
----------------------------------------
补偿量的统计指标：
count    30.000000
mean      0.751059
std       0.245066
min       0.123573
25%       0.592138
50%       0.843780
75%       0.912945
max       0.999912
dtype: float64
补偿量占预测偏差量百分比的统计指标：
count     30.0
mean     100.0
std        0.0
min   

In [13]:
# 2. 对于第二大类即补偿变差的情况

m21, m22 = len(y_s2_c1), len(y_s2_c2)  # 同侧补偿且变差
m23, m24 = len(y_s2_c3), len(y_s2_c4)  # 对侧补偿且变差
m25, m26 = len(y_s2_c5), len(y_s2_c6)  # 完全准确预测
m2syn, m2opp, m2com = m21+m22, m23+m14, m25+m26  # 补偿变差时同侧、对侧、完全准确预测的序列数
if m21+m22+m23+m24+m25+m26 == m2:
    x21, x22, x23, x24, x25, x26 = m21/m2*100, m22/m2*100, m23/m2*100, m24/m2*100, m25/m2*100, m26/m2*100
    x2syn, x2opp, x2com = m2syn/m2*100, m2opp/m2*100, m2com/m2*100
    print('计算正确，当补偿变差时，其下6种子情况的序列总数相加与补偿变差的序列总数相等：\n')
    print('预测销量与补偿后销量均比真实销量低，补偿变差且发生在同侧，该种情况有：{:.0f}个序列，\
占补偿变差序列数的比例为：{:.2f}%'.format(m21, x21))
    print('预测销量与补偿后销量均比真实销量高，补偿变差且发生在同侧，该种情况有：{:.0f}个序列，\
占补偿变差序列数的比例为：{:.2f}%'.format(m22, x22))
    print('预测销量偏低，补偿后销量偏高，补偿变差且发生在对侧，该种情况有：{:.0f}个序列，\
占补偿变差序列数的比例为：{:.2f}%'.format(m23, x23))
    print('预测销量偏高，补偿后销量偏低，补偿变差且发生在对侧，该种情况有：{:.0f}个序列，\
占补偿变差序列数的比例为：{:.2f}%'.format(m24, x24))
    print('完全准确预测，补偿后销量偏低，该种情况有：{:.0f}个序列，占补偿变差序列数的比例为：{:.2f}%'.format(m25, x25))
    print('完全准确预测，补偿后销量偏高，该种情况有：{:.0f}个序列，占补偿变差序列数的比例为：{:.2f}%'.format(m26, x26))
    print('补偿变差且发生在同侧的序列数有：{:.0f}个，占补偿变差序列数的比例为：{:.2f}%'.format(m2syn, x2syn))
    print('补偿变差且发生在对侧的序列数有：{:.0f}个，占补偿变差序列数的比例为：{:.2f}%'.format(m2opp, x2opp))
    print('完全准确预测的序列数有：{:.0f}个，占补偿变差序列数的比例为：{:.2f}%'.format(m2com, x2com))  
else: raise Exception('计算有误，当补偿变差时，其下6种子情况的序列总数相加与补偿变差的序列总数不等。')

计算正确，当补偿变差时，其下6种子情况的序列总数相加与补偿变差的序列总数相等：

预测销量与补偿后销量均比真实销量低，补偿变差且发生在同侧，该种情况有：582个序列，占补偿变差序列数的比例为：23.58%
预测销量与补偿后销量均比真实销量高，补偿变差且发生在同侧，该种情况有：616个序列，占补偿变差序列数的比例为：24.96%
预测销量偏低，补偿后销量偏高，补偿变差且发生在对侧，该种情况有：605个序列，占补偿变差序列数的比例为：24.51%
预测销量偏高，补偿后销量偏低，补偿变差且发生在对侧，该种情况有：612个序列，占补偿变差序列数的比例为：24.80%
完全准确预测，补偿后销量偏低，该种情况有：24个序列，占补偿变差序列数的比例为：0.97%
完全准确预测，补偿后销量偏高，该种情况有：29个序列，占补偿变差序列数的比例为：1.18%
补偿变差且发生在同侧的序列数有：1198个，占补偿变差序列数的比例为：48.54%
补偿变差且发生在对侧的序列数有：1211个，占补偿变差序列数的比例为：49.07%
完全准确预测的序列数有：53个，占补偿变差序列数的比例为：2.15%


In [14]:
# 2.1 预测补偿均低，补偿同侧变差：
print('预测补偿均低，补偿同侧变差时：\n')

pva_s2_c1 = abs(y_s2_c1 - yhat_s2_c1)  # 预测偏差量
pva_s2_c1_sts = pva_s2_c1.describe()
pvr_s2_c1 = abs(pva_s2_c1[y_s2_c1!=0] / y_s2_c1[y_s2_c1!=0])*100  # 预测偏差量与真实销量百分比
pvr_s2_c1_sts = pvr_s2_c1.describe()
print('预测偏差量的统计指标：')
print(pva_s2_c1_sts)
print('预测偏差量与真实销量百分比的统计指标：')
print(pvr_s2_c1_sts)
print('----------------------------------------')

cva_s2_c1 = abs(y_s2_c1 - ycomp_s2_c1)  # 补偿后剩余偏差量
cva_s2_c1_sts = cva_s2_c1.describe()
cvr_s2_c1 = abs(cva_s2_c1[y_s2_c1!=0] / y_s2_c1[y_s2_c1!=0])*100  # 补偿后剩余偏差量与真实销量百分比
cvr_s2_c1_sts = cvr_s2_c1.describe()
print('补偿后剩余偏差量的统计指标：')
print(cva_s2_c1_sts)
print('补偿后剩余偏差量与真实销量百分比的统计指标：')
print(cvr_s2_c1_sts)
print('----------------------------------------')

ca_s2_c1 = abs(yhat_s2_c1 - ycomp_s2_c1)  # 补偿量
ca_s2_c1_sts = ca_s2_c1.describe()
car_s2_c1 = abs(ca_s2_c1[pva_s2_c1!=0] / pva_s2_c1[pva_s2_c1!=0])*100  # 补偿量占预测偏差量百分比
car_s2_c1_sts = car_s2_c1.describe()
print('补偿量的统计指标：')
print(ca_s2_c1_sts)
print('补偿量占预测偏差量百分比的统计指标：')
print(car_s2_c1_sts)
print('----------------------------------------')

if (len(pva_s2_c1)==len(cva_s2_c1)==len(ca_s2_c1)==len(car_s2_c1)==m21) & (len(pva_s2_c1)>=len(pvr_s2_c1)) & (len(cva_s2_c1)>=len(cvr_s2_c1)): # 按位与运算符(&)的优先级高于比较运算符和等于运算符，所以需用()括起每个条件
    print('在2.1预测补偿均低，补偿同侧变差的情况中，各分量的长度匹配正确。')
else: raise Exception('在2.1即预测补偿均低，补偿同侧变差的情况中，各分量的长度匹配存在错误，请检查该异常。')


预测补偿均低，补偿同侧变差时：

预测偏差量的统计指标：
count    582.000000
mean       0.484626
std        0.287052
min        0.008851
25%        0.262375
50%        0.444113
75%        0.750987
max        0.999755
dtype: float64
预测偏差量与真实销量百分比的统计指标：
count    543.000000
mean       8.407922
std       14.234100
min        0.020117
25%        1.533694
50%        3.748212
75%        8.803781
max       95.375265
dtype: float64
----------------------------------------
补偿后剩余偏差量的统计指标：
count    582.000000
mean       0.827251
std        0.201125
min        0.132352
25%        0.750987
50%        0.905578
75%        0.985146
max        0.999990
dtype: float64
补偿后剩余偏差量与真实销量百分比的统计指标：
count    543.000000
mean      14.196324
std       19.701172
min        0.429311
25%        3.309685
50%        6.773444
75%       16.042017
max       99.975517
dtype: float64
----------------------------------------
补偿量的统计指标：
count    582.000000
mean       0.342624
std        0.257953
min        0.000235
25%        0.124818
50%        0.280783
7

In [15]:
# 2.2 预测补偿均高，补偿同侧变差：
print('预测补偿均高，补偿同侧变差时：\n')

pva_s2_c2 = abs(y_s2_c2 - yhat_s2_c2)  # 预测偏差量
pva_s2_c2_sts = pva_s2_c2.describe()
pvr_s2_c2 = abs(pva_s2_c2[y_s2_c2!=0] / y_s2_c2[y_s2_c2!=0])*100  # 预测偏差量与真实销量百分比
pvr_s2_c2_sts = pvr_s2_c2.describe()
print('预测偏差量的统计指标：')
print(pva_s2_c2_sts)
print('预测偏差量与真实销量百分比的统计指标：')
print(pvr_s2_c2_sts)
print('----------------------------------------')

cva_s2_c2 = abs(y_s2_c2 - ycomp_s2_c2)  # 补偿后剩余偏差量
cva_s2_c2_sts = cva_s2_c2.describe()
cvr_s2_c2 = abs(cva_s2_c2[y_s2_c2!=0] / y_s2_c2[y_s2_c2!=0])*100  # 补偿后剩余偏差量与真实销量百分比
cvr_s2_c2_sts = cvr_s2_c2.describe()
print('补偿后剩余偏差量的统计指标：')
print(cva_s2_c2_sts)
print('补偿后剩余偏差量与真实销量百分比的统计指标：')
print(cvr_s2_c2_sts)
print('----------------------------------------')

ca_s2_c2 = abs(yhat_s2_c2 - ycomp_s2_c2)  # 补偿量
ca_s2_c2_sts = ca_s2_c2.describe()
car_s2_c2 = abs(ca_s2_c2[pva_s2_c2!=0] / pva_s2_c2[pva_s2_c2!=0])*100  # 补偿量占预测偏差量百分比
car_s2_c2_sts = car_s2_c2.describe()
print('补偿量的统计指标：')
print(ca_s2_c2_sts)
print('补偿量占预测偏差量百分比的统计指标：')
print(car_s2_c2_sts)
print('----------------------------------------')

if (len(pva_s2_c2)==len(cva_s2_c2)==len(ca_s2_c2)==len(car_s2_c2)==m22) & (len(pva_s2_c2)>=len(pvr_s2_c2)) & (len(cva_s2_c2)>=len(cvr_s2_c2)): # 按位与运算符(&)的优先级高于比较运算符和等于运算符，所以需用()括起每个条件
    print('在2.2即预测补偿均高，补偿同侧变差的情况中，各分量的长度匹配正确。')
else: raise Exception('在2.2即预测补偿均高，补偿同侧变差的情况中，各分量的长度匹配存在错误，请检查该异常。')


预测补偿均高，补偿同侧变差时：

预测偏差量的统计指标：
count    616.000000
mean       0.434182
std        0.281920
min        0.017702
25%        0.167356
50%        0.404022
75%        0.656987
max        0.992873
dtype: float64
预测偏差量与真实销量百分比的统计指标：
count    574.000000
mean       7.843941
std       13.051443
min        0.022127
25%        1.226253
50%        3.023403
75%        8.212332
max       95.637593
dtype: float64
----------------------------------------
补偿后剩余偏差量的统计指标：
count    616.000000
mean       0.813512
std        0.215611
min        0.035398
25%        0.733190
50%        0.901788
75%        0.983588
max        0.999912
dtype: float64
补偿后剩余偏差量与真实销量百分比的统计指标：
count    574.000000
mean      15.291920
std       21.082295
min        0.187347
25%        3.446806
50%        7.229016
75%       16.529961
max       99.952016
dtype: float64
----------------------------------------
补偿量的统计指标：
count    616.000000
mean       0.379330
std        0.262055
min        0.001249
25%        0.148622
50%        0.343058
7

In [16]:
# 2.3 预测偏低补偿偏高，补偿对侧变差：
print('预测偏低补偿偏高，补偿对侧变差时：\n')

pva_s2_c3 = abs(y_s2_c3 - yhat_s2_c3)  # 预测偏差量
pva_s2_c3_sts = pva_s2_c3.describe()
pvr_s2_c3 = abs(pva_s2_c3[y_s2_c3!=0] / y_s2_c3[y_s2_c3!=0])*100  # 预测偏差量与真实销量百分比
pvr_s2_c3_sts = pvr_s2_c3.describe()
print('预测偏差量的统计指标：')
print(pva_s2_c3_sts)
print('预测偏差量与真实销量百分比的统计指标：')
print(pvr_s2_c3_sts)
print('----------------------------------------')

cva_s2_c3 = abs(y_s2_c3 - ycomp_s2_c3)  # 补偿后剩余偏差量
cva_s2_c3_sts = cva_s2_c3.describe()
cvr_s2_c3 = abs(cva_s2_c3[y_s2_c3!=0] / y_s2_c3[y_s2_c3!=0])*100  # 补偿后剩余偏差量与真实销量百分比
cvr_s2_c3_sts = cvr_s2_c3.describe()
print('补偿后剩余偏差量的统计指标：')
print(cva_s2_c3_sts)
print('补偿后剩余偏差量与真实销量百分比的统计指标：')
print(cvr_s2_c3_sts)
print('----------------------------------------')

ca_s2_c3 = abs(yhat_s2_c3 - ycomp_s2_c3)  # 补偿量
ca_s2_c3_sts = ca_s2_c3.describe()
car_s2_c3 = abs(ca_s2_c3[pva_s2_c3!=0] / pva_s2_c3[pva_s2_c3!=0])*100  # 补偿量占预测偏差量百分比
car_s2_c3_sts = car_s2_c3.describe()
print('补偿量的统计指标：')
print(ca_s2_c3_sts)
print('补偿量占预测偏差量百分比的统计指标：')
print(car_s2_c3_sts)
print('----------------------------------------')

if (len(pva_s2_c3)==len(cva_s2_c3)==len(ca_s2_c3)==len(car_s2_c3)==m23) & (len(pva_s2_c3)>=len(pvr_s2_c3)) & (len(cva_s2_c3)>=len(cvr_s2_c3)): # 按位与运算符(&)的优先级高于比较运算符和等于运算符，所以需用()括起每个条件
    print('在2.3即预测偏低补偿偏高，补偿对侧变差的情况中，各分量的长度匹配正确。')
else: raise Exception('在2.3即预测偏低补偿偏高，补偿对侧变差的情况中，各分量的长度匹配存在错误，请检查该异常。')


预测偏低补偿偏高，补偿对侧变差时：

预测偏差量的统计指标：
count    605.000000
mean       0.457607
std        0.288028
min        0.008851
25%        0.245252
50%        0.428183
75%        0.676772
max        0.999207
dtype: float64
预测偏差量与真实销量百分比的统计指标：
count    554.000000
mean       8.476263
std       15.126909
min        0.013830
25%        1.187161
50%        3.115044
75%        8.517673
max       99.177885
dtype: float64
----------------------------------------
补偿后剩余偏差量的统计指标：
count    605.000000
mean       0.813392
std        0.207896
min        0.017702
25%        0.683262
50%        0.901788
75%        0.968364
max        0.999912
dtype: float64
补偿后剩余偏差量与真实销量百分比的统计指标：
count    554.000000
mean      14.077974
std       20.613873
min        0.398623
25%        3.289862
50%        6.570567
75%       14.538825
max       99.991186
dtype: float64
----------------------------------------
补偿量的统计指标：
count    605.000000
mean       1.270999
std        0.429358
min        0.026553
25%        0.965688
50%        1.289570

In [17]:
# 2.4 预测偏高补偿偏低，补偿对侧变差：
print('预测偏高补偿偏低，补偿对侧变差时：\n')

pva_s2_c4 = abs(y_s2_c4 - yhat_s2_c4)  # 预测偏差量
pva_s2_c4_sts = pva_s2_c4.describe()
pvr_s2_c4 = abs(pva_s2_c4[y_s2_c4!=0] / y_s2_c4[y_s2_c4!=0])*100  # 预测偏差量与真实销量百分比
pvr_s2_c4_sts = pvr_s2_c4.describe()
print('预测偏差量的统计指标：')
print(pva_s2_c4_sts)
print('预测偏差量与真实销量百分比的统计指标：')
print(pvr_s2_c4_sts)
print('----------------------------------------')

cva_s2_c4 = abs(y_s2_c4 - ycomp_s2_c4)  # 补偿后剩余偏差量
cva_s2_c4_sts = cva_s2_c4.describe()
cvr_s2_c4 = abs(cva_s2_c4[y_s2_c4!=0] / y_s2_c4[y_s2_c4!=0])*100  # 补偿后剩余偏差量与真实销量百分比
cvr_s2_c4_sts = cvr_s2_c4.describe()
print('补偿后剩余偏差量的统计指标：')
print(cva_s2_c4_sts)
print('补偿后剩余偏差量与真实销量百分比的统计指标：')
print(cvr_s2_c4_sts)
print('----------------------------------------')

ca_s2_c4 = abs(yhat_s2_c4 - ycomp_s2_c4)  # 补偿量
ca_s2_c4_sts = ca_s2_c4.describe()
car_s2_c4 = abs(ca_s2_c4[pva_s2_c4!=0] / pva_s2_c4[pva_s2_c4!=0])*100  # 补偿量占预测偏差量百分比
car_s2_c4_sts = car_s2_c4.describe()
print('补偿量的统计指标：')
print(ca_s2_c4_sts)
print('补偿量占预测偏差量百分比的统计指标：')
print(car_s2_c4_sts)
print('----------------------------------------')

if (len(pva_s2_c4)==len(cva_s2_c4)==len(ca_s2_c4)==len(car_s2_c4)==m24) & (len(pva_s2_c4)>=len(pvr_s2_c4)) & (len(cva_s2_c4)>=len(cvr_s2_c4)): # 按位与运算符(&)的优先级高于比较运算符和等于运算符，所以需用()括起每个条件
    print('在2.4即预测偏高补偿偏低，补偿对侧变差的情况中，各分量的长度匹配正确。')
else: raise Exception('在2.4即预测偏高补偿偏低，补偿对侧变差的情况中，各分量的长度匹配存在错误，请检查该异常。')


预测偏高补偿偏低，补偿对侧变差时：

预测偏差量的统计指标：
count    612.000000
mean       0.470156
std        0.291536
min        0.017702
25%        0.253823
50%        0.420167
75%        0.733190
max        0.999912
dtype: float64
预测偏差量与真实销量百分比的统计指标：
count    574.000000
mean       7.526537
std       12.804280
min        0.045390
25%        1.226985
50%        3.240057
75%        8.180706
max       99.060736
dtype: float64
----------------------------------------
补偿后剩余偏差量的统计指标：
count    612.000000
mean       0.817177
std        0.211488
min        0.026551
25%        0.750987
50%        0.905578
75%        0.985146
max        0.999990
dtype: float64
补偿后剩余偏差量与真实销量百分比的统计指标：
count    574.000000
mean      13.386780
std       18.498381
min        0.106205
25%        3.500929
50%        7.077682
75%       14.190874
max       99.999021
dtype: float64
----------------------------------------
补偿量的统计指标：
count    612.000000
mean       1.287333
std        0.433847
min        0.044253
25%        0.996098
50%        1.308389

In [18]:
# 2.5 完全准确预测，补偿偏低：
print('完全准确预测，补偿偏低：\n')

pva_s2_c5 = abs(y_s2_c5 - yhat_s2_c5)  # 预测偏差量
pva_s2_c5_sts = pva_s2_c5.describe()
pvr_s2_c5 = abs(pva_s2_c5[y_s2_c5!=0] / y_s2_c5[y_s2_c5!=0])*100  # 预测偏差量与真实销量百分比
pvr_s2_c5_sts = pvr_s2_c5.describe()
if ((pva_s2_c5==0).sum() == len(pva_s2_c5)) & ((pvr_s2_c5==0).sum() == len(pvr_s2_c5)):
    print('预测偏差量的统计指标：')
    print(pva_s2_c5_sts)
    print('预测偏差量与真实销量百分比的统计指标：')
    print(pvr_s2_c5_sts)
    print('----------------------------------------')
else: raise Exception('2.5 在第二大类的第五小类中，即完全准确预测补偿偏低这种情况下，\
预测偏差量(pva_s2_c5)和/或预测偏差比(pvr_s2_c5)存在计算有误的情况，请检查错误原因。')

cva_s2_c5 = abs(y_s2_c5 - ycomp_s2_c5)  # 补偿后剩余偏差量
cva_s2_c5_sts = cva_s2_c5.describe()
cvr_s2_c5 = abs(cva_s2_c5[y_s2_c5!=0] / y_s2_c5[y_s2_c5!=0])*100  # 补偿后剩余偏差量与真实销量百分比
cvr_s2_c5_sts = cvr_s2_c5.describe()
print('补偿后剩余偏差量的统计指标：')
print(cva_s2_c5_sts)
print('补偿后剩余偏差量与真实销量百分比的统计指标：')
print(cvr_s2_c5_sts)
print('----------------------------------------')

ca_s2_c5 = abs(yhat_s2_c5 - ycomp_s2_c5)  # 补偿量
ca_s2_c5_sts = ca_s2_c5.describe()
car_s2_c5_reciprocal = pva_s2_c5/ca_s2_c5  # 因为此处补偿量占预测偏差量百分比为无穷大，所以采用其倒数，即预测偏差量占补偿量百分比，其结果均为0，避免报错
car_s2_c5_reciprocal_sts = car_s2_c5_reciprocal.describe()
if ((ca_s2_c5==cva_s2_c5).sum() == len(ca_s2_c5)) & ((car_s2_c5_reciprocal==0).sum() == len(car_s2_c5_reciprocal)):
    print('补偿量的统计指标：')
    print(ca_s2_c5_sts)
    print('预测偏差量占补偿量百分比的统计指标：')
    print(car_s2_c5_reciprocal_sts)
    print('----------------------------------------')
else: raise Exception('2.5 在第二大类的第五小类中，即预测完全准确，补偿偏低这种情况下，\
补偿量(ca_s2_c5)、补偿比的倒数(car_s2_c5_reciprocal)、补偿后剩余偏差量(cva_s2_c5)中存在计算有误的情况，请检查错误原因。')

if (len(pva_s2_c5)==len(cva_s2_c5)==len(ca_s2_c5)==len(car_s2_c5_reciprocal)==m25) & (len(pva_s2_c5)>=len(pvr_s2_c5)) & (len(cva_s2_c5)>=len(cvr_s2_c5)): # 按位与运算符(&)的优先级高于比较运算符和等于运算符，所以需用()括起每个条件
    print('在2.5即完全准确预测，补偿偏低的情况中，各分量的长度匹配正确。')
else: raise Exception('在2.5即完全准确预测，补偿偏低的情况中，各分量的长度匹配存在错误，请检查该异常。')


完全准确预测，补偿偏低：

预测偏差量的统计指标：
count    24.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
dtype: float64
预测偏差量与真实销量百分比的统计指标：
count    21.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
dtype: float64
----------------------------------------
补偿后剩余偏差量的统计指标：
count    24.000000
mean      0.695035
std       0.290324
min       0.114785
25%       0.526137
50%       0.794289
75%       0.929664
max       0.999755
dtype: float64
补偿后剩余偏差量与真实销量百分比的统计指标：
count    21.000000
mean     15.221739
std      21.022035
min       1.432792
25%       3.433562
50%       8.295424
75%      15.862267
max      84.622040
dtype: float64
----------------------------------------
补偿量的统计指标：
count    24.000000
mean      0.695035
std       0.290324
min       0.114785
25%       0.526137
50%       0.794289
75%       0.929664
max       0.999755
dtype: float64
预测偏差量占补偿量百分比的统计指标：
count    24.0
mean      0.0
std       0.0
min     

In [19]:
# 2.6 完全准确预测，补偿偏高：
print('完全准确预测，补偿偏高：\n')

pva_s2_c6 = abs(y_s2_c6 - yhat_s2_c6)  # 预测偏差量
pva_s2_c6_sts = pva_s2_c6.describe()
pvr_s2_c6 = abs(pva_s2_c6[y_s2_c6!=0] / y_s2_c6[y_s2_c6!=0])*100  # 预测偏差量与真实销量百分比
pvr_s2_c6_sts = pvr_s2_c6.describe()
if ((pva_s2_c6==0).sum() == len(pva_s2_c6)) & ((pvr_s2_c6==0).sum() == len(pvr_s2_c6)):
    print('预测偏差量的统计指标：')
    print(pva_s2_c6_sts)
    print('预测偏差量与真实销量百分比的统计指标：')
    print(pvr_s2_c6_sts)
    print('----------------------------------------')
else: raise Exception('2.5 在第二大类的第六小类中，即完全准确预测补偿偏高这种情况下，\
预测偏差量(pva_s2_c6)和/或预测偏差比(pvr_s2_c6)存在计算有误的情况，请检查错误原因。')

cva_s2_c6 = abs(y_s2_c6 - ycomp_s2_c6)  # 补偿后剩余偏差量
cva_s2_c6_sts = cva_s2_c6.describe()
cvr_s2_c6 = abs(cva_s2_c6[y_s2_c6!=0] / y_s2_c6[y_s2_c6!=0])*100  # 补偿后剩余偏差量与真实销量百分比
cvr_s2_c6_sts = cvr_s2_c6.describe()
print('补偿后剩余偏差量的统计指标：')
print(cva_s2_c6_sts)
print('补偿后剩余偏差量与真实销量百分比的统计指标：')
print(cvr_s2_c6_sts)
print('----------------------------------------')

ca_s2_c6 = abs(yhat_s2_c6 - ycomp_s2_c6)  # 补偿量
ca_s2_c6_sts = ca_s2_c6.describe()
car_s2_c6_reciprocal = pva_s2_c6/ca_s2_c6  # 因为此处补偿量占预测偏差量百分比为无穷大，所以采用其倒数，即预测偏差量占补偿量百分比，其结果均为0，避免报错
car_s2_c6_reciprocal_sts = car_s2_c6_reciprocal.describe()
if ((ca_s2_c6==cva_s2_c6).sum() == len(ca_s2_c6)) & ((car_s2_c6_reciprocal==0).sum() == len(car_s2_c6_reciprocal)):
    print('补偿量的统计指标：')
    print(ca_s2_c6_sts)
    print('预测偏差量占补偿量百分比的统计指标：')
    print(car_s2_c6_reciprocal_sts)
    print('----------------------------------------')
else: raise Exception('2.6 在第二大类的第六小类中，即预测完全准确，补偿偏高这种情况下，\
补偿量(ca_s2_c6)、补偿比的倒数(car_s2_c6_reciprocal)、补偿后剩余偏差量(cva_s2_c6)中存在计算有误的情况，请检查错误原因。')

if (len(pva_s2_c6)==len(cva_s2_c6)==len(ca_s2_c6)==len(car_s2_c6_reciprocal)==m26) & (len(pva_s2_c6)>=len(pvr_s2_c6)) & (len(cva_s2_c6)>=len(cvr_s2_c6)): # 按位与运算符(&)的优先级高于比较运算符和等于运算符，所以需用()括起每个条件
    print('在2.6即完全准确预测，补偿偏高的情况中，各分量的长度匹配正确。')
else: raise Exception('在2.6即完全准确预测，补偿偏高的情况中，各分量的长度匹配存在错误，请检查该异常。')


完全准确预测，补偿偏高：

预测偏差量的统计指标：
count    29.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
dtype: float64
预测偏差量与真实销量百分比的统计指标：
count    28.0
mean      0.0
std       0.0
min       0.0
25%       0.0
50%       0.0
75%       0.0
max       0.0
dtype: float64
----------------------------------------
补偿后剩余偏差量的统计指标：
count    29.000000
mean      0.777260
std       0.228950
min       0.149877
25%       0.650288
50%       0.826829
75%       0.986628
max       0.999912
dtype: float64
补偿后剩余偏差量与真实销量百分比的统计指标：
count    28.000000
mean      7.626421
std      12.441762
min       0.599509
25%       2.425483
50%       5.244958
75%       6.219320
max      63.673801
dtype: float64
----------------------------------------
补偿量的统计指标：
count    29.000000
mean      0.777260
std       0.228950
min       0.149877
25%       0.650288
50%       0.826829
75%       0.986628
max       0.999912
dtype: float64
预测偏差量占补偿量百分比的统计指标：
count    29.0
mean      0.0
std       0.0
min     